## Импортируем библиотеки, создаём функцию для препроцессинга текстов

In [292]:
!pip install yake
!pip3 install summa
!pip3 install keybert
import pandas as pd
import numpy as np
import pymorphy2
import re
import yake
from summa import keywords
from keybert import KeyBERT
kw_model = KeyBERT('clips/mfaq')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
kw_extractor = yake.KeywordExtractor()
morph = pymorphy2.MorphAnalyzer()


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\transformers\configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [293]:
def preprocessing(text, stop_words):
    morph = pymorphy2.MorphAnalyzer() #лемматизируем с помощью pymorphy2
    known_words = {} #этот словарь позволит нам не лемматизировать одно и то же слово два раза
    lemmas = [] #сюда кладем все леммы в тексте
    regex_text = re.sub('[^\w\s]', '', text)
    regex_text = re.sub('[\d]', '', regex_text) #чистим текст от пунктуации, переносов строк и чисел
    regex_text = re.sub('\n', ' ', regex_text)
    for word in regex_text.split(' '):
        if word in stop_words: #добавляем леммы в словарь со встретившимися словами и в список лемм
            continue 
        elif word in known_words:
            lemmas.append(known_words[word])
        else:
            lemma = morph.parse(word.lower())[0].normal_form
            lemmas.append(lemma)
            known_words[word] = lemma
    res = re.sub('  ', ' ', ' '.join(lemmas)) #заменяем двойные пробелы на одинарные и соединяем леммы в строку
    return res

## Работаем с мини-корпусом

Подготовил мини-корпус из 16 новостей с сайта ferra.ru, на котором ключевые слова представлены как теги

In [294]:
data = pd.read_excel('texts_with_keywords.xlsx')

Лемматизируем тексты новостей

In [295]:
for ind, line in enumerate(data['text']):
    data['text'].loc[ind] = preprocessing(line, stop_words)

In [296]:
data

,text,given_keywords,my_keywords
0,депутат госдума антон горелкин заявить пользов...,"Google, иск, Государственная Дума, Россия","Государственная Дума, слежка, Google, иск, суд..."
1,голливудский кинокомпания warner bros запретит...,"санкции, Warner Bros, Первый канал, СТС, Гарри...","Warner Bros, фильмы, Национальная медиа группа..."
2,физический упражнение известно приносить множе...,"спорт, здоровье, человек, Сердце, Инсульт, мед...","упражнения, организм, сердце, инсульт, спорт"
3,группа исследователь больница ветеран тайбэя и...,"AirPods, наушники, слуховой аппарат","Apple, AirPods, Live Listen, слуховой аппарат,..."
4,сбермаркет разработать серия простой миниигра ...,"СберМаркет, акции, промо-коды","СберМаркет, игры, промо-коды, акции,"
5,ранее пользователь новый флагманский видеокарт...,"NVIDIA RTX 4090, GEFORCE, NVIDIA","Nvidia, GeForce RTX 4090, Gamers Nexus, перегрев"
6,иногда бывать сложно понять какой продукт дейс...,"здоровье, медицина, продукты","продукты, организм, польза, маркетинг, здоровье"
7,японский подразделение nintendo обновить свой ...,"NINTENDO, ремонт","Nintendo, обслуживание, ремонт, грубость"
8,если думать рак лёгкий указывать кашель знать ...,"рак, здоровье, человек, Легкие","рак, легкие, здоровье, симптомы, инфекция, бол..."
9,в браузер google chrome появиться функция кото...,"GOOGLE CHROME, браузер","Google, Google Chrome, вкладки, браузер, функция"


In [297]:
given_keywords = [set(list_of_keywords.lower().split(', ')) for list_of_keywords in data['given_keywords']]
my_keywords = [set(list_of_keywords.lower().split(', ')) for list_of_keywords in data['my_keywords']]

In [298]:
print(given_keywords)
print(my_keywords)

[{'google', 'россия', 'иск', 'государственная дума'}, {'санкции', 'гарри поттер', 'кино', 'фильмы', 'warner bros', 'стс', 'первый канал'}, {'человек', 'сердце', 'медицина', 'спорт', 'инсульт', 'здоровье'}, {'airpods', 'наушники', 'слуховой аппарат'}, {'промо-коды', 'акции', 'сбермаркет'}, {'geforce', 'nvidia rtx 4090', 'nvidia'}, {'медицина', 'здоровье', 'продукты'}, {'nintendo', 'ремонт'}, {'легкие', 'рак', 'здоровье', 'человек'}, {'браузер', 'google chrome'}, {'microsoft', 'skype', 'день в истории'}, {'fujitsu', 'жёсткий диск', 'hdd', 'ноутбук', 'день в истории'}, {'sony playstation', 'playstation', 'микрофон', 'клон', 'sony', 'sony playstation 2', 'день в истории'}, {'видео', 'россия', 'автопилот', 'автомобиль', 'беспилотник'}, {'xiaomi 12', 'xiaomi', 'скидки и распродажи', 'смартфон'}, {'видео', 'игровой компьютер', 'nintendo switch', 'steam deck', 'игровая консоль', 'steam', 'игровой ноутбук'}]
[{'иск', 'google', 'слежка', 'депутат', 'суд', 'государственная дума'}, {'гарри потер',

## Создаем золотой стандарт ключевых слов

In [299]:
data['golden'] = ''
for ind in range(len(given_keywords)):
    golden_set = given_keywords[ind].intersection(my_keywords[ind])
    data['golden'].loc[ind] = ', '.join(list(golden_set))

In [300]:
data['golden']

0                     google, иск, государственная дума
1                фильмы, warner bros, стс, первый канал
2                                сердце, спорт, инсульт
3                             airpods, слуховой аппарат
4                         промо-коды, акции, сбермаркет
5                                                nvidia
6                                    здоровье, продукты
7                                      nintendo, ремонт
8                                 легкие, рак, здоровье
9                                браузер, google chrome
10                     microsoft, skype, день в истории
11       fujitsu, жёсткий диск, день в истории, ноутбук
12                       клон, sony, sony playstation 2
13                       автопилот, россия, беспилотник
14                                     xiaomi, смартфон
15    steam, nintendo switch, steam deck, игровая ко...
Name: golden, dtype: object

## Получаем ключевые слова с помощью трех методов извлечения слов

In [301]:
data['yake_keywords'] = '' # Заменил Rake на YAKE, так как Rake не выделял ключевые слова в некоторых текстах
data['textrank_keywords'] = ''
data['keybert_keywords'] = ''
for ind, line in enumerate(data['text']):
    yake_extractor = yake.KeywordExtractor(lan='ru', n=2, top=7, features=None)
    yake_keywords = [tup[0] for tup in yake_extractor.extract_keywords(line)]
    textrank_keywords = [tup[0] for tup in keywords.keywords(line, language='russian', additional_stopwords=stop_words, scores=True)]
    keybert_keywords = [tup[0] for tup in kw_model.extract_keywords(line)]
    data['yake_keywords'].loc[ind] = ', '.join(yake_keywords)
    data['textrank_keywords'].loc[ind] = ', '.join(textrank_keywords)
    data['keybert_keywords'].loc[ind] = ', '.join(keybert_keywords)

In [302]:
data

,text,given_keywords,my_keywords,golden,yake_keywords,textrank_keywords,keybert_keywords
0,депутат госдума антон горелкин заявить пользов...,"Google, иск, Государственная Дума, Россия","Государственная Дума, слежка, Google, иск, суд...","google, иск, государственная дума","горелкин заявить, депутат госдума, госдума ант...","пользователь, слежка google, мир, глобальный, ...","госдума, государство, иск, спор, заявить"
1,голливудский кинокомпания warner bros запретит...,"санкции, Warner Bros, Первый канал, СТС, Гарри...","Warner Bros, фильмы, Национальная медиа группа...","фильмы, warner bros, стс, первый канал","bros запретить, warner bros, запретить трансли...","транслировать свой фильм канал, warner, стс, в...","moretv, tv, телевидение, транслировать, киноко..."
2,физический упражнение известно приносить множе...,"спорт, здоровье, человек, Сердце, Инсульт, мед...","упражнения, организм, сердце, инсульт, спорт","сердце, спорт, инсульт","болезнь сердце, ишемический болезнь, развитие ...","активность, утро, исследование, который, риск ...","упражнение, физический, заниматься, особенно, ..."
3,группа исследователь больница ветеран тайбэя и...,"AirPods, наушники, слуховой аппарат","Apple, AirPods, Live Listen, слуховой аппарат,...","airpods, слуховой аппарат","слуховой аппарат, шумный обстановка, модель ai...","airpods функция, оказаться устройство, разный ...","изучить, эксперимент, устройство, слух, listen"
4,сбермаркет разработать серия простой миниигра ...,"СберМаркет, акции, промо-коды","СберМаркет, игры, промо-коды, акции,","промо-коды, акции, сбермаркет","сбермаркет сити, сити вообще, сити представлят...","магазин, сбермаркет, получить, задание, игра н...","миниигра, игровой, игра, сбермаркет, простой"
5,ранее пользователь новый флагманский видеокарт...,"NVIDIA RTX 4090, GEFORCE, NVIDIA","Nvidia, GeForce RTX 4090, Gamers Nexus, перегрев",nvidia,"gamers nexus, разъём питание, ранее пользовате...","разъём питание, проблема, gamers nexus, пользо...","проблема, сбой, затем, привести, описывать"
6,иногда бывать сложно понять какой продукт дейс...,"здоровье, медицина, продукты","продукты, организм, польза, маркетинг, здоровье","здоровье, продукты","дешёвый масло, фруктовый сок, фруктоза фруктов...","бывать сложно понять, специя считаться, самост...","полезный, food, пищевой, организм, семя"
7,японский подразделение nintendo обновить свой ...,"NINTENDO, ремонт","Nintendo, обслуживание, ремонт, грубость","nintendo, ремонт","японский подразделение, позиция эффективный, о...","nintendo, сотрудник, клиент, случай, компания,...","обновить, ремонт, обслуживать, предупредить, о..."
8,если думать рак лёгкий указывать кашель знать ...,"рак, здоровье, человек, Легкие","рак, легкие, здоровье, симптомы, инфекция, бол...","легкие, рак, здоровье","рак лёгкий, неделя долгий, потеря аппетит, дли...","это, кашель, который, рак лёгкий, долгий, поте...","симптом, кашель, боль, заболевание, рак"
9,в браузер google chrome появиться функция кото...,"GOOGLE CHROME, браузер","Google, Google Chrome, вкладки, браузер, функция","браузер, google chrome","google chrome, группа вкладка, сохранить групп...","функция, браузер, нужный, нужно, группа вкладк...","браузер, функция, tab, закладка, включить"


## Работа с шаблонами

In [303]:
templates = [['NOUN', 'NOUN'], ['ADJF', 'NOUN'], ['NOUN', 'VERB'], ['VERB', 'NOUN'], ['NOUN']] # Добавил простое существительное, так как иначе лишаемся большого количества ключевых слов в textrank и keybert
def filtering(keywords):
    filtered = []
    for keyword in keywords.split(', '):
        if [morph.parse(word)[0].tag.POS for word in keyword.split()] in templates or [str(morph.parse(word)[0].tag) for word in keyword.split()] == ['LATN']: # Оставляем английские слова
            filtered.append(keyword)
    return ', '.join(filtered)

In [304]:
data['golden_filtered'] = ''
data['yake_keywords_filtered'] = ''
data['textrank_keywords_filtered'] = ''
data['keybert_keywords_filtered'] = ''
for ind in range(len(data['text'])):
    data['golden_filtered'].loc[ind] = filtering(data['golden'].loc[ind])
    data['yake_keywords_filtered'].loc[ind] = filtering(data['yake_keywords'].loc[ind])
    data['textrank_keywords_filtered'].loc[ind] = filtering(data['textrank_keywords'].loc[ind])
    data['keybert_keywords_filtered'].loc[ind] = filtering(data['keybert_keywords'].loc[ind])

In [305]:
data

,text,given_keywords,my_keywords,golden,yake_keywords,textrank_keywords,keybert_keywords,golden_filtered,yake_keywords_filtered,textrank_keywords_filtered,keybert_keywords_filtered
0,депутат госдума антон горелкин заявить пользов...,"Google, иск, Государственная Дума, Россия","Государственная Дума, слежка, Google, иск, суд...","google, иск, государственная дума","горелкин заявить, депутат госдума, госдума ант...","пользователь, слежка google, мир, глобальный, ...","госдума, государство, иск, спор, заявить","google, иск, государственная дума","депутат госдума, госдума антон","пользователь, мир, иск, горелкин, политик","госдума, государство, иск, спор"
1,голливудский кинокомпания warner bros запретит...,"санкции, Warner Bros, Первый канал, СТС, Гарри...","Warner Bros, фильмы, Национальная медиа группа...","фильмы, warner bros, стс, первый канал","bros запретить, warner bros, запретить трансли...","транслировать свой фильм канал, warner, стс, в...","moretv, tv, телевидение, транслировать, киноко...","фильмы, стс, первый канал","национальный медиа, медиа группа, голливудский...","warner, стс","moretv, tv, телевидение, кинокомпания"
2,физический упражнение известно приносить множе...,"спорт, здоровье, человек, Сердце, Инсульт, мед...","упражнения, организм, сердце, инсульт, спорт","сердце, спорт, инсульт","болезнь сердце, ишемический болезнь, развитие ...","активность, утро, исследование, который, риск ...","упражнение, физический, заниматься, особенно, ...","сердце, спорт, инсульт","болезнь сердце, ишемический болезнь, инсульт б...","активность, утро, исследование, риск инсульт, ...",упражнение
3,группа исследователь больница ветеран тайбэя и...,"AirPods, наушники, слуховой аппарат","Apple, AirPods, Live Listen, слуховой аппарат,...","airpods, слуховой аппарат","слуховой аппарат, шумный обстановка, модель ai...","airpods функция, оказаться устройство, разный ...","изучить, эксперимент, устройство, слух, listen","airpods, слуховой аппарат","слуховой аппарат, шумный обстановка, группа ис...","разный модель, доллар, слуховой аппарат, pro, ...","эксперимент, устройство, слух, listen"
4,сбермаркет разработать серия простой миниигра ...,"СберМаркет, акции, промо-коды","СберМаркет, игры, промо-коды, акции,","промо-коды, акции, сбермаркет","сбермаркет сити, сити вообще, сити представлят...","магазин, сбермаркет, получить, задание, игра н...","миниигра, игровой, игра, сбермаркет, простой","промо-коды, акции, сбермаркет","сбермаркет сити, лапа сердце, простой миниигра","магазин, сбермаркет, задание, заказ, сити","миниигра, игра, сбермаркет"
5,ранее пользователь новый флагманский видеокарт...,"NVIDIA RTX 4090, GEFORCE, NVIDIA","Nvidia, GeForce RTX 4090, Gamers Nexus, перегрев",nvidia,"gamers nexus, разъём питание, ранее пользовате...","разъём питание, проблема, gamers nexus, пользо...","проблема, сбой, затем, привести, описывать",nvidia,разъём питание,"разъём питание, проблема, пользователь, корпус...","проблема, сбой"
6,иногда бывать сложно понять какой продукт дейс...,"здоровье, медицина, продукты","продукты, организм, польза, маркетинг, здоровье","здоровье, продукты","дешёвый масло, фруктовый сок, фруктоза фруктов...","бывать сложно понять, специя считаться, самост...","полезный, food, пищевой, организм, семя","здоровье, продукты","дешёвый масло, фруктовый сок","зерно, перец, корень, исследование, мелкий кус...","food, организм, семя"
7,японский подразделение nintendo обновить свой ...,"NINTENDO, ремонт","Nintendo, обслуживание, ремонт, грубость","nintendo, ремонт","японский подразделение, позиция эффективный, о...","nintendo, сотрудник, клиент, случай, компания,...","обновить, ремонт, обслуживать, предупредить, о...","nintendo, ремонт","японский подразделение, свой условие, условие ...","nintendo, сотрудник, клиент, случай, компания,...","ремонт, обслуживание"
8,если думать рак лёгкий указывать кашель знать ...,"рак, здоровье, человек, Легкие","рак, легкие, здоровье, симптомы, инфекция, бол...","легкие, рак, здоровье","рак лёгкий, неделя

## Оценка

In [306]:
scores = {}
for i in data[['yake_keywords', 'textrank_keywords', 'keybert_keywords', 'yake_keywords_filtered', 'textrank_keywords_filtered', 'keybert_keywords_filtered']]:
    if i.endswith('filtered'):
        standart = data['golden_filtered']
    else:
        standart = data['golden']
    received = data[i]
    precision = [len(set(standart[elem].split(', ')).intersection(set(received[elem].split(', ')))) / len(received[elem].split(', ')) for elem in range(len(standart))]
    recall = [len(set(standart[elem].split(', ')).intersection(set(received[elem].split(', ')))) / len(standart[elem].split(', ')) for elem in range(len(standart))]
    f1_mean = [2 * precision[elem] * recall[elem] / (precision[elem] + recall[elem]) if precision[elem] + recall[elem] != 0 else 0 for elem in range(len(precision))]
    scores[i] = np.mean(precision), np.mean(recall), np.mean(f1_mean)
data_scores = np.transpose(pd.DataFrame(scores))
data_scores.columns = ['precision', 'recall', 'f1_mean']

In [307]:
data_scores

,precision,recall,f1_mean
yake_keywords,0.053571,0.125000,0.073232
textrank_keywords,0.068158,0.296875,0.103400
keybert_keywords,0.087500,0.161458,0.112103
yake_keywords_filtered,0.036458,0.052083,0.041667
textrank_keywords_filtered,0.126701,0.333333,0.164191
keybert_keywords_filtered,0.132292,0.208333,0.151488


Моя фильтрация сумела повысить все метрики у большинства методов извлечения слов. Единственный метод извлечения слов, который упал в метриках precision, recall и f1_mean - YAKE. К YAKE у меня и до оценки были вопросы, главным из которых является непонятное мне желание этого метода создавать большое количество составных ключевых слов, большинство из которых по крайней мере в первом приближении ключевыми не являются. В идеале я бы заменил YAKE, но если продолжать работу именно с ним, я бы, возможно, сделал некоторое ограничение на порождения им ключевых слов - кажется, падение связано именно с этим.